In [1]:
import pandas as pd
import numpy as np

In [24]:
def get_instrument(df):
    volume_sum = df['volume'].groupby(df['instrument_id']).sum().sort_values(ascending=False)
    return volume_sum.idxmax()

In [ ]:
datapath = '../data/ES/glbx-mdp3-20170601.ohlcv-1m.csv'
# Fucntion Aggregates the data into 5 minute bars
def process_file(datapath):
    df = pd.read_csv(datapath, index_col='ts_event')
    df.index = pd.to_datetime(df.index, unit='ns')
    price_columns = ['open', 'high', 'low', 'close']
    df[price_columns] = df[price_columns].apply(lambda x: x /10**9)
    instrument_id = get_instrument(df)
    filtered_df = df[df['instrument_id'] == instrument_id] # Most used instrument by day and volume
    return filtered_df.resample('5T').agg({'open': 'first', 
                                           'high': 'max',
                                           'low': 'min',
                                           'close': 'last',
                                           'volume': 'sum'})

In [25]:
# Testing the function
datapath = '../data/ES/glbx-mdp3-20170601.ohlcv-1m.csv'
df = pd.read_csv(datapath, index_col='ts_event')
df.index = pd.to_datetime(df.index, unit='ns')
price_columns = ['open', 'high', 'low', 'close']
df[price_columns] = df[price_columns].apply(lambda x: x /10**9)
instrument_id = get_instrument(df)
filtered_df = df[df['instrument_id'] == instrument_id] # Most used instrument by day and volume
regular_df = filtered_df.asfreq('T')
resampled_df = df.resample('5T').agg({'open': 'first', 
                                       'high': 'max',
                                       'low': 'min',
                                       'close': 'last',
                                       'volume': 'sum'})
missing_bars = regular_df[regular_df.isnull().any(axis=1)]
resampled_df

,open,high,low,close,volume
ts_event,,,,,
2017-06-01 05:00:00,2413.25,2413.50,2410.25,2413.00,260
2017-06-01 05:05:00,2413.00,2413.25,2413.00,2413.00,30
2017-06-01 05:10:00,2413.00,2413.25,2412.50,2412.50,155
2017-06-01 05:15:00,2412.50,2412.75,2412.25,2412.50,84
2017-06-01 05:20:00,2412.25,2412.75,2409.75,2412.25,173
...,...,...,...,...,...
2017-06-01 23:35:00,2429.25,2429.50,2426.75,2429.50,149
2017-06-01 23:40:00,2427.00,2429.50,2427.00,2429.25,53
2017-06-01 23:45:00,2429.50,2429.75,-2.45,2429.50,189


In [31]:
# processing many csv files into one dataframe then save to csv
import glob
import os
import pandas as pd
# Replace this path with the path to your CSV files

path_to_csv_files = '../data/ES/'
all_files = [os.path.join(path_to_csv_files, f) for f in os.listdir(path_to_csv_files) if 'ohlcv-1m.csv' in f]

# Process each file and aggregate
all_dataframes = [process_file(file) for file in all_files]
combined_df = pd.concat(all_dataframes)


2002